In [46]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
data_path = 'D:\\jupyter files\\data_waiting_time_project\\preprocess_data\\'

In [3]:
data_part1 = pd.read_csv(data_path + 'data_part1.csv', index_col = 0)
data_part2 = pd.read_csv(data_path + 'data_part2.csv', index_col = 0)

C:\Users\WENDY\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [16]:
data_part1_grouped = data_part1.iloc[: 10000].groupby('AppointmentSerNum')

appt_one_list = []
appt_more_list = []
for appt in data_part1_grouped.groups.keys():
    sample = data_part1_grouped.get_group(appt)
    if sample.shape[0] == 1:
        appt_one_list.append(appt)
print(f'{len(appt_one_list)} with one appointment')

appt_more_list = [i for i in data_part1_grouped.groups.keys() if i not in appt_one_list]
print(f'{len(appt_more_list)} with more appointment')


# key_list = list(data_part1_grouped.groups.keys())
data_part1_new = pd.DataFrame({})
for i in range(len(appt_more_list)):
    if (i + 1) % 500 == 0:
        print(f'\n{i+1} appointment data')
        
    sample = data_part1_grouped.get_group(appt_more_list[i])
    new_appt = pd.DataFrame({
        'PatientSerNum': sample.PatientSerNum.tolist()[0],
        'AppointmentSerNum': sample.AppointmentSerNum.tolist()[0],
        'Sex': sample.Sex.tolist()[0],
        'age': sample.age.tolist()[0],
        'month': sample.month.tolist()[0],
        'date': sample.date.tolist()[0],
        'week': sample.week.tolist()[0],
        'hour': sample.hour.tolist()[0],
        'Scheduled_duration': sample.Scheduled_duration.tolist()[0],
        'Actual_duration': sample.Actual_duration.tolist()[0],
        'ScheduledStartTime': sample.ScheduledStartTime.tolist()[0],
        'ScheduledEndTime': sample.ScheduledEndTime.tolist()[0],
        'ActualStartDate': sample.ActualStartDate.tolist()[0],
        'ActualEndDate': sample.ActualEndDate.tolist()[0],
        
        'dxt_AliasName': [list(set(sample.dxt_AliasName.tolist()))], 
        'AliasSerNum': [list(set(sample.AliasSerNum.tolist()))], 
        'CourseSerNum': [list(set(sample.CourseSerNum.tolist()))], 
        'PlanSerNum': [list(set(sample.PlanSerNum.tolist()))], 
        'TreatmentOrientation': [list(set(sample.TreatmentOrientation.tolist()))],
        
    })
    
    data_part1_new = pd.concat([data_part1_new, new_appt], axis = 0)
data_part1_new.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime'], inplace = True)
data_part1_new.reset_index(drop = True, inplace = True)

457 with one appointment
1900 with more appointment

500 appointment data

1000 appointment data

1500 appointment data


In [27]:
data_part2_grouped = data_part2.iloc[: 10000].groupby(['PatientSerNum', 'date'])

treat_one_list = []
treat_more_list = []
for k in data_part2_grouped.groups.keys():
    sample = data_part2_grouped.get_group(k)
    if sample.shape[0] == 1:
        treat_one_list.append(appt)
print(f'{len(treat_one_list)} with one treatment')

treat_more_list = [i for i in data_part2_grouped.groups.keys() if i not in treat_one_list]
print(f'{len(treat_more_list)} with more treatment')

# key_list = list(data_part2_grouped.groups.keys())
data_part2_new = pd.DataFrame({})
for i in range(len(treat_more_list)):
    if (i+1) % 500 == 0:
        print(f'\n{i+1} treatment data')
        
    sample = data_part2_grouped.get_group(treat_more_list[i])
    try:
        new_treat = pd.DataFrame({
            'PatientSerNum': sample.PatientSerNum.tolist()[0],
            'FractionNumber': sample.FractionNumber.tolist()[0],
            'date': sample.date.tolist()[0],

            'UserName': [list(set(sample.UserName.tolist()))], 
            'RadiationSerNum': [list(set(sample.RadiationSerNum.tolist()))], 
            'RadiationId': [list(set(sample.RadiationId.tolist()))], 
            'ResourceSerNum': [list(set(sample.ResourceSerNum.tolist()))], 
            'CourseId': [list(set(sample.CourseId.tolist()))],

            'ImagesTaken_total': sum(sample.ImagesTaken.tolist()),
            'MU_total': sum(sample.MU.tolist()),
            'MUCoeff_total': sum(sample.MUCoeff.tolist()),
            'TreatmentTime_total': sum(sample.TreatmentTime.tolist()),
        })
        data_part2_new = pd.concat([data_part2_new, new_treat], axis = 0)
    except:
        print(treat_more_list[i])
    
    
data_part2_new.sort_values(by = ['PatientSerNum', 'date', 'FractionNumber'], inplace = True)
data_part2_new.reset_index(drop = True, inplace = True)

16 with one treatment
3117 with more treatment

500 treatment data

1000 treatment data

1500 treatment data

2000 treatment data

2500 treatment data

3000 treatment data


In [32]:
data_part1_new.head()
print(f'The shape of data_part1_new is {data_part1_new.shape}')
print(f'The shape of data_part1_new is {data_part2_new.shape}')

DATA = pd.merge(data_part1_new, data_part2_new, on = ['PatientSerNum', 'date'], how = 'inner')
DATA.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
print(f'The shape of data is {DATA.shape}')

The shape of data_part1_new is (1900, 19)
The shape of data_part1_new is (3117, 12)
The shape of data is (584, 29)


In [40]:
DATA_grouped = DATA.groupby('PatientSerNum')

series_data = []
for pat in DATA_grouped.groups.keys():
    if DATA_grouped.get_group(pat).shape[0] != 1:
        series_data.append(pat)
print(f'series data {len(series_data)}')

series_data = pd.DataFrame({'PatientSerNum': series_data})
series_data = pd.merge(series_data, DATA, on = 'PatientSerNum', how = 'inner')
series_data.sort_values(by = ['ScheduledStartTime'], inplace = True)
series_data.head()

series data 52


,PatientSerNum,AppointmentSerNum,Sex,age,month,date,week,hour,Scheduled_duration,Actual_duration,...,FractionNumber,UserName,RadiationSerNum,RadiationId,ResourceSerNum,CourseId,ImagesTaken_total,MU_total,MUCoeff_total,TreatmentTime_total
470,4524,570195,Male,68,1,2015--01--05,1,9,15.0,14.0,...,24,[poon],"[402821, 402822]","[1.2 ARC2, 1.1 ARC1]",[95],[C1],0,420.815,200.0,1.666666
471,4524,570196,Male,68,1,2015--01--06,2,9,15.0,15.0,...,25,[poon],"[402821, 402822]","[1.2 ARC2, 1.1 ARC1]",[95],[C1],0,420.815,200.0,1.683333
169,1809,568788,Male,67,1,2015--01--06,2,14,15.0,48.0,...,1,[Chen Shi Lei],"[409004, 409012, 409013, 409005]","[1.4ARC2, 1.2ARC2, 1.1ARC1, 1.3ARC1]",[107],[C1],14,4580.170,400.0,7.950000
473,4524,570198,Male,68,1,2015--01--07,3,9,15.0,15.0,...,1,[Basdeo Reeta],"[408554, 408555]","[2.2 ARC2, 2.1 ARC1]",[95],[C1],2,449.769,200.0,1.683333
170,1809,568789,Male,67,1,2015--01--07,3,14,30.0,22.0,...,2,[Stachura Jacob],"[409012, 409013]","[1.4ARC2, 1.3ARC1]",[107],[C1],4,1041.980,200.0,1.900000


In [41]:
series_data.columns

Index(['PatientSerNum', 'AppointmentSerNum', 'Sex', 'age', 'month', 'date',
       'week', 'hour', 'Scheduled_duration', 'Actual_duration',
       'ScheduledStartTime', 'ScheduledEndTime', 'ActualStartDate',
       'ActualEndDate', 'dxt_AliasName', 'AliasSerNum', 'CourseSerNum',
       'PlanSerNum', 'TreatmentOrientation', 'FractionNumber', 'UserName',
       'RadiationSerNum', 'RadiationId', 'ResourceSerNum', 'CourseId',
       'ImagesTaken_total', 'MU_total', 'MUCoeff_total',
       'TreatmentTime_total'],
      dtype='object')

In [45]:
# Index(['PatientSerNum', 'AppointmentSerNum',  
#        'ScheduledStartTime', 'ScheduledEndTime', 
#        'ActualStartDate', 'ActualEndDate',
#        'CourseSerNum', 'PlanSerNum', 'RadiationSerNum'

feature_num = ['Scheduled_duration', 'Actual_duration', 'Actual_duration',
               'age', 'TreatmentTime_total', 'ImagesTaken_total',
               'MU_total', 'MUCoeff_total']

feature_cate = ['dxt_AliasName', 'Sex', 'AliasSerNum', 
                'month', 'week', 'hour', 'DoctorSerNum', 
                'TreatmentOrientation', 'FractionNumber',
                'UserName', 'RadiationId', 'CourseId', 'ResourceSerNum']

# 因为数据缺失，所以添加新特征，表示是否是连续的序列

In [ ]:
data_cate = series_data[feature_cate]
data_num = series_data[feature_num]



In [47]:
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()# sparse = False

In [53]:
data1 = data_part1.iloc[: 10000]
data2 = data_part2.iloc[: 10000]

encode_data = pd.merge(data1, data2, on = ['PatientSerNum', 'date'], how = 'inner')
encode_data.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
encode_data = encode_data[feature_cate]
print(f'The shape of encode_data is {encode_data.shape}')

The shape of encode_data is (13254, 13)


In [54]:
integer_encoded = label_encoder.fit_transform(encode_data.Sex)
integer_encoded

array([1, 1, 1, ..., 0, 0, 0])

In [58]:
inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[0, :])])
print(inverted)

['Male']


In [56]:
# one-hot encode
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
onehot_encoded

C:\Users\WENDY\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


<13254x2 sparse matrix of type '<class 'numpy.float64'>'
	with 13254 stored elements in Compressed Sparse Row format>